In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter

In [3]:
# 1. PDF 파일 로드
# 
loader = PyMuPDFLoader('./book2.pdf')  # Document loaders인 PyMuPDFLoader를 사용해서 PDF 파일을 읽을 준비를 한다
documents = loader.load()              # load 메서드를 호출하여 PDF 파일을 문서 형식으로 가져온다.

# load() 메소드는 문장의 배열을 리턴한다.
# 하나의 문장을 Document라는 클래스로 표현한다.
# PyMuPDFLoader에서는 1페이지 마다 1개의 Document가 생성된다.
# PyMuPDFLoader에서는 하나의 PDF 파일을 로드하면 몇 페이지 분량의 Document가 배열로 리턴된다.

In [4]:
documents

[Document(metadata={'source': './book2.pdf', 'file_path': './book2.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.4', 'title': '', 'author': 'User', 'subject': '', 'keywords': '', 'creator': '한컴오피스 한글 2018', 'producer': 'Hancom PDF 1.3.0.515', 'creationDate': "D:20240524003422+09'00'", 'modDate': "D:20240524003422+09'00'", 'trapped': ''}, page_content='# 500 \n## 공학\n- 초보자를 위한 파이썬(Python) 200제\n- 파이썬으로 시작하는 데이터 분석\n- 누구나 할 수 있다 파이썬 Python 기초\n- OpenAI API와 파이썬으로 나만의 챗GPT 만들기\n# 600 \n## 문학\n- 현직 교사가 직접 고른 생기부 고전 필독서 30: 한국문학 편\n- EBS 수능완성 사용설명서 독서·문학(2024)(2025 수능대비)\n# 800 \n## 역사\n- 2025 전한길 한국사 합격생 필기노트\n- 설쌤과 놀면서 학습하는 한국사 대모험 스티커 색칠놀이\n')]

In [8]:
# 2. 텍스트 분할기 초기화
#
# 텍스트가 너무 길면 모델이 처리할 수 있는 글자수의 한계를 넘계되어서 텍스트를 분할해야한다.
# 텍스트 분할기를 만든다. 텍스트를 분할하기 위한 준비.
text_splitter = SpacyTextSplitter(		
    chunk_size=300, 					# 분할할 때 각 청크의 최대 길이를 300자로 설정
    pipeline='ko_core_news_sm'			# 분할에 사용할 한국어 모델인 ko_core_news_sm
)

# spaCy는 자연어 처리 라이브러리.
# 자연어 처리는 인간에서 사용하는 언어를 컴퓨터가 이해하고 분석할 수 있도록 하는 기술로
# spaCy 에는 문장 분할, 품사 판단, 명사구 추출, 구문 분석 등을 제공한다.

In [9]:
# 3. 문서 분할
# 
# text_splitter를 사용하여 로드된 문서를 여러 조각으로 나눈다.
# 입력으로 받은 documents를 내부적으로 설정된 방식(여기서는 스페이시 한국어 모델과 최대 300자의 청크 크기)을 사용하여 분할합니다.
splitted_documents = text_splitter.split_documents(documents)

In [10]:
splitted_documents

[Document(metadata={'source': './book2.pdf', 'file_path': './book2.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.4', 'title': '', 'author': 'User', 'subject': '', 'keywords': '', 'creator': '한컴오피스 한글 2018', 'producer': 'Hancom PDF 1.3.0.515', 'creationDate': "D:20240524003422+09'00'", 'modDate': "D:20240524003422+09'00'", 'trapped': ''}, page_content='# 500 \n## 공학\n- 초보자를 위한 파이썬(Python) 200제\n\n\n- 파이썬으로 시작하는 데이터 분석\n- 누구나 할 수 있다 파이썬 Python 기초\n- OpenAI API와 파이썬으로 나만의 챗GPT 만들기\n\n\n# 600 \n## 문학\n- 현직 교사가 직접 고른 생기부 고전 필독서 30: 한국문학 편\n- EBS 수능완성 사용설명서 독서·문학(2024)(2025 수능대비)\n\n\n# 800 \n## 역사\n- 2025 전한길 한국사 합격생 필기노트\n- 설쌤과 놀면서 학습하는 한국사 대모험 스티커 색칠놀이')]

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [12]:
# 4. 임베딩 생성기 초기화
# OpenAI의 text-embedding-ada-002 모델을 사용하여 텍스트 임베딩을 생성하고, 이것을 벡터 스토어에 저장하기 위한 설정
embeddings = OpenAIEmbeddings(
    openai_api_key=key,					# OpenAI API 키를 사용하여 인증
    model='text-embedding-ada-002' 		# OpenAI의 text-embedding-ada-002 모델을 사용하여 텍스트 임베딩을 생성
)

d:\kodefare_sample\gpt_speak_lib_app\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
# 5. 벡터 스토어 초기화. 
#
# Chroma 벡터 스토어를 초기화하고, 임베딩 데이터를 ./data 디렉토리에 저장합니다.
# 벡터 스토어는 텍스트 임베딩을 저장하고 검색하는 데 사용됩니다.
database = Chroma(							# Chroma 벡터 스토어를 초기화
    persist_directory = './data', 			# 임베딩 데이터를 저장할 디렉토리를 ./data로 설정
    embedding_function = embeddings			# embedding 함수로 위에서 만든 embeddings 객체 지정
)

# 이 과정은 긴 텍스트를 처리하고 검색 가능하도록 임베딩으로 변환하여 저장하는 데 유용합니다. 
# 벡터 스토어를 통해 효율적인 검색과 데이터 관리가 가능해집니다.

In [15]:
#  6. 분할된 문서를 벡터 스토어에 저장
# 
# database는 Chroma 벡터 스토어 객체이다.
# add_documents() 메서드는 분할된 텍스트 조각을 벡터 스토어에 추가하는 역할
# splitted_documents는 앞서 분할된 텍스트 조각들의 리스트입니다. 이 리스트의 각 항목은 텍스트 청크

# 각 텍스트 청크를 벡터로 변환하고, 변환된 벡터를 벡터 스토어에 저장

database.add_documents(			# 분할된 텍스트 조각들을 데이터베이스에 추가한다
    splitted_documents			# 추가할 문서를 지정
)

print('완료')

완료
